In [ ]:
%load_ext autoreload 
%autoreload 2

%matplotlib inline

In [ ]:
from src.lights import change_lights
from src.main import generate_scenario, run_scenario
from src.logger import ExperimentLogger

from utils.common import do_test_steps, create_simulator, get_camera_matrix, place_agent

from concurrent.futures import ThreadPoolExecutor

In [ ]:
def create_experiment(sim_settings_filename):
    logger = ExperimentLogger(sim_settings_filename)
    sim_settings, _ = logger.get_settings()

    sim = create_simulator(sim_settings)

    return {'logger': logger, 'sim': sim}


def run_experiment(experiment, bar_inds=[0, 1], display=False):
    logger = experiment['logger']
    sim = experiment['sim']

    sim_settings, light_settings = logger.get_settings()

    start_point, navigatable_points = generate_scenario(sim, sim_settings, display=display)

    sim = place_agent(sim, sim_settings['default_agent'], start_point)

    camera_matrix = get_camera_matrix(sim.agents[sim_settings['default_agent']])
    logger.save_camera_params(camera_matrix)

    run_scenario(sim, sim_settings, light_settings, navigatable_points, logger, display=display, bar_inds=bar_inds)

In [ ]:
config_count = 5
num_workers = min(config_count, 12)

experiments = [create_experiment(f'sim_settings_{i}.json') for i in range(config_count)]

In [ ]:
for i, exp in enumerate(experiments):
    run_experiment(exp, [i*2, i*2 + 1])

In [ ]:
!rm generated.zip
!zip -r generated.zip generated/